In [13]:
import os
import json
import re
import pandas as pd
import csv
import collections
import numpy as np
from ipyfilechooser import FileChooser
from plotnine import ggplot, geom_point, geom_line, aes, geom_label, coord_cartesian, themes, ylim
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display
import random
import subprocess

In [2]:
# Create and display a FileChooser widget
fc = FileChooser('/home/breck/git/codatmo/red_team_blue_team/output')
display(fc)

FileChooser(path='/home/breck/git/codatmo/red_team_blue_team/output', filename='', title='HTML(value='', layou…

In [36]:
#@interact(num_samples=widgets.IntSlider(min=10, max=1000, step=10, value=10),
#          verbose = False)


def load_stan_run(num_samples, verbose, stan_summary):
    data = json.load(open(f"{fc.selected_path}/data.json"))
    config = json.load(open(f"{fc.selected_path}/config.json"))

    stan_files = [f"{fc.selected_path}/{f}" for f in os.listdir(fc.selected_path) if f.endswith('csv')]

    fit = pd.DataFrame()
    last_index = 0
    sig_digits= 2
    summary = None
    if os.path.exists(f"{fc.selected_path}/out.1.csv"):
        if not os.path.exists(f"{fc.selected_path}/stan_summary.csv"):
            if verbose:
                    print('Creating stan_summary.csv')
            cmd = (f"/home/breck/.cmdstanr/cmdstan-2.27.0/bin/stansummary {fc.selected_path}/*.csv " +
                   f" -s {sig_digits} -c {fc.selected_path}/stan_summary.csv")
            returned_value = subprocess.call(cmd, shell=True)
        if verbose:
            print("loading stan_summary")
        summary = pd.read_csv(f"{fc.selected_path}/stan_summary.csv", comment='#')

        for stan_csv in stan_files:
            if stan_csv.endswith("stan_summary.csv"):
                continue
            if verbose:
                print(stan_csv)
            chain = pd.read_csv(stan_csv, nrows=num_samples, comment='#')
            chain['index'] = range(last_index, last_index + len(chain))
            last_index = last_index + len(chain)
            fit = fit.append(chain)
        if verbose:
            print(fit.shape)
    else:
        if verbose:
            print("No data from Stan run seen")
    RunData = collections.namedtuple('RunData', 'config data fit summary')
    rd = RunData(config, data, fit, summary) 
    return(rd)
        
rd1 = interactive(load_stan_run, {'manual': True}, num_samples=widgets.IntSlider(min=10, max=1000, step=10, value=10),
          verbose = False, stan_summary = True)

display(rd1)




interactive(children=(IntSlider(value=10, description='num_samples', max=1000, min=10, step=10), Checkbox(valu…

In [37]:
print(rd1.result.summary)


               name          Mean          MCSE        StdDev            5%  \
0              lp__  1.500000e+03  6.600000e+02  9.400000e+02  6.000000e+02   
1     accept_stat__  9.000000e-01  4.400000e-03  1.300000e-01  6.300000e-01   
2        stepsize__  1.800000e-02  6.400000e-03  9.100000e-03  7.000000e-03   
3       treedepth__  6.200000e+00  2.700000e-01  1.600000e+00  3.000000e+00   
4      n_leapfrog__  1.600000e+02  5.000000e+01  1.400000e+02  1.200000e+01   
...             ...           ...           ...           ...           ...   
5260         D[287]  1.729340e+08  1.874505e+06  2.658053e+06  1.702670e+08   
5261         D[288]  1.732081e+08  1.901001e+06  2.695526e+06  1.705040e+08   
5262         D[289]  1.734798e+08  1.927431e+06  2.732907e+06  1.707380e+08   
5263         D[290]  1.737490e+08  1.953809e+06  2.770218e+06  1.709700e+08   
5264         D[291]  1.740159e+08  1.980134e+06  2.807451e+06  1.711990e+08   

               50%           95%  N_Eff  N_Eff/s   

In [4]:
def graph_coefs(plot, rd, num_samples):
    fit = rd.fit
    if fit is None:
        print("No Stan output to graph coefs with")
        return(plot)
    draws = fit.sample(n=num_samples)
    variables = []
    values = []
    draw = []
    vars_2_report = ['iDay1_est', 'beta', 'gamma', 'deathRate', 'lambda_twitter']
    for value in vars_2_report:
        variables.extend([value] * len(draws))
        values.extend(draws[value])
        draw.extend(draws['index'])

    df = pd.DataFrame({'variable' :  variables,
                      'value' : values,
                      'index' : draw})
    df['variable'] = pd.Categorical(df['variable'], categories=vars_2_report, 
                                    ordered=True)
    print(df)
    return(plot + geom_line(data=df, 
                     mapping=(aes(x='variable', y='value', group='index')))
          )

def graph_predictions(plot, rd, num_samples):
    fit = rd.fit
    if fit is None:
        print("No Stan output to graph predictions with")
        return(plot)
    data = rd.data
    sample = fit.sample(n=num_samples)
    time = []
    draws = []
    values = []
    variables = []
    vars_2_report = ['pred_tweets', 'pred_deaths']
    for var in vars_2_report:
        for t in range(1,292):
            values.extend(sample[f'{var}.{t}'])
            time.extend([t]*num_samples)
            variables.extend([var]*num_samples)
            draws.extend(range(0,num_samples))
    df = pd.DataFrame({'variable' : variables,
                       'draws' :  draws,
                       'value' : values,
                       'time' : time})
    plot = plot + geom_line(data=df[df['variable'] == 'pred_deaths'], 
                    mapping=(aes(x='time', y='value', group='draws')))
    plot = plot + geom_line(data=df[df['variable'] == 'pred_tweets'], 
              mapping=(aes(x='time', y='value', group='draws')))
    return(plot)

def graph_sim_states(plot, rd):
    config = rd.config
    if 's' not in config:
        print("No simulation output to graph states with")
        return(plot)
    time = []
    draws = []
    values = []
    variables = []
    vars_2_report = ['s', 'i', 'r', 'd', 'tweets']
#    vars_2_report = ['i']
    for var in vars_2_report:
        time.extend(range(1,config['n_days'] + 1))
        variables.extend([var]*config['n_days'])
        values.extend(config[var][0])
    df = pd.DataFrame({'variable' : variables,
                       'value' : values,
                       'time' : time})
    #print(df)
    plot = plot + geom_line(data=df, 
                    mapping=(aes(x='time', y='value', color='variable')))
    return(plot)

def graph_data(plot, rd):
    data = rd.data
    time = []
    draws = []
    values = []
    variables = []
    vars_2_report = ['tweets', 'deaths']
    for var in vars_2_report:
        time.extend(range(1,data['n_days'] + 1))
        variables.extend([var]*data['n_days'])
        values.extend(data[var])
        #for t in range(1,data.n_days):
        #    values.extend(sample[f'{var}.{t}'])
        #    time.extend([t]*num_samples)
        #    variables.extend([var]*num_samples)
        #    draws.extend(range(0,num_samples))
    df = pd.DataFrame({'variable' : variables,
                       'value' : values,
                       'time' : time})
    plot = plot + geom_line(data=df, 
                    mapping=(aes(x='time', y='value', color='variable')))
    return(plot)

def graph_ODE(plot, rd, num_samples):
    fit = rd.fit
    if fit is None:
        print("No Stan output to graph ODE with")
        return(plot)
    config = rd.config
    sample = fit.sample(n=num_samples)
    time = []
    draws = []
    values = []
    variables = []
    vars_2_report = ['pred_tweets', 'pred_deaths']
    for var in vars_2_report:
        for t in range(1,292):
            values.extend(sample[f'{var}.{t}'])
            time.extend([t]*num_samples)
            variables.extend([var]*num_samples)
            draws.extend(range(0,num_samples))
    df = pd.DataFrame({'variable' : variables,
                       'draws' :  draws,
                       'value' : values,
                       'time' : time})
    plot = plot + geom_line(data=df[df['variable'] == 'pred_deaths'], 
                    mapping=(aes(x='time', y='value', group='draws')))
    plot = plot + geom_line(data=df[df['variable'] == 'pred_tweets'], 
              mapping=(aes(x='time', y='value', group='draws')))
    return(plot)


    

In [5]:
rd1.result.config['i'][0]


[3419,
 3408.72410979083,
 3398.47258776313,
 3388.24541943388,
 3378.04259000088,
 3367.86408434518,
 3357.7098870336,
 3347.57998232121,
 3337.47435415383,
 3327.39298617047,
 3317.33586170579,
 3307.30296379257,
 3297.29427516414,
 3287.30977825681,
 3277.3494552123,
 3267.41328788014,
 3257.50125782013,
 3247.61334630465,
 3237.74953432113,
 3227.90980257437,
 3218.09413148897,
 3208.30250121162,
 3198.53489161351,
 3188.79128229267,
 3179.07165257625,
 3169.37598152292,
 3159.70424792512,
 3150.05643031141,
 3140.43250694877,
 3130.83245584485,
 3121.25625475027,
 3111.7038811609,
 3102.17531232012,
 3092.67052522106,
 3083.18949660883,
 3073.73220298278,
 3064.29862059873,
 3054.88872547112,
 3045.50249337532,
 3036.13989984971,
 3026.80092019797,
 3017.48552949117,
 3008.19370256999,
 2998.92541404688,
 2989.68063830817,
 2980.45934951624,
 2971.26152161164,
 2962.0871283152,
 2952.93614313016,
 2943.80853934427,
 2934.70429003184,
 2925.62336805591,
 2916.5657460702,
 2907.5313

In [40]:
@interact(rd = fixed(rd1.result))
def sim_coefs(rd):
    if rd.config is None:
        return("No config to print parameters from")
    df = pd.DataFrame(rd.config)
    return(df[['beta_mean','gamma', 'death_prob', 'tweet_rate', 'n_patient_zero']])
    

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [49]:
@interact(rd = fixed(rd1.result),
          add_coefs = True)
def summarize(rd, add_coefs):
    if rd.summary is None:
        return("No summary to print parameters from")
    summary = rd.summary
    return(summary[summary['name'].isin(['beta', 'gamma', 'deathRate', 'lambda_twitter',
                                         'iDay1_est', 'sdDeaths_r'])])
    

interactive(children=(Checkbox(value=True, description='add_coefs'), Output()), _dom_classes=('widget-interact…

In [35]:
@interact(run_d = fixed(rd1.result), 
          ylimit=widgets.IntSlider(min=1, max=100000000, step=10000, value=200000),
          add_sim_states = True,
          add_coefs = False,
          add_preds = False,
          ribbon = False,
          add_data = False,
          add_ODE = False)
def graph(run_d, ylimit, add_sim_states, add_coefs, add_preds, ribbon, add_data, add_ODE):
   plot = (ggplot())
   if add_sim_states:
       plot = graph_sim_states(plot=plot, rd=run_d)
   if add_coefs:
      plot = graph_coefs(plot=plot, rd=run_d, num_samples=10)
   if add_preds:
      plot = graph_predictions(plot=plot, rd=run_d, num_samples=10)
   if add_data:
      plot = graph_data(plot=plot, rd=run_d)
   if add_ODE:
      plot = graph_ODE(plot=plot, rd=run_d, num_samples=10)
   print(plot + coord_cartesian(ylim=(0, ylimit)))
           
      #  geom_point(size=0.25) +
      #  geom_line(data=, mapping=(aes(x='day', y='count')), color='red') +
      #  geom_line(data=df_long_brazil_tweets, mapping=(aes(x='day', y='count')), color='blue') +
      # geom_label(data = df_label, mapping=aes(label='label')) +
      
        
       # ))
      #coord_cartesian(ylim=(0, max_y),xlim=(0,n_days + 50))
    #graph_sim/graph_data
    #graph_ode
    #graph_predictions
    

interactive(children=(IntSlider(value=200000, description='ylimit', max=100000000, min=1, step=10000), Checkbo…

In [ ]:
#' Graph internal state counts for simulation and corresponding tweets for a run
#' of the runEval framework. Returns a ggplot geom_point element with x = days
#' y = count.
#' @param data_df one row of the run_df with simulation data added
#' @param hide_s Boolean to control whether to hide the s or susceptible counts
def graph_sim(config, plot) {
    if ('t' in config) { 
      sim_df = data.frame(day = 1:data_df$n_days, 
                        tweets = unlist(data_df$tweets), 
                        s = unlist(data_df$s),
                        i = unlist(data_df$i),
                        r = unlist(data_df$r),
                        t = unlist(data_df$t),
                        d = unlist(data_df$d))

      compartment_names <- c('s', 'i', 'r', 't', 'd')
    }
    else {
       sim_df = data.frame(day = 1:data_df$n_days, 
                        tweets = unlist(data_df$tweets), 
                        s = unlist(data_df$s),
                        i = unlist(data_df$i),
                        r = unlist(data_df$r),
                        d = unlist(data_df$d))
       compartment_names <- c('s', 'i', 'r', 'd')
    }
    if (hide_s) {
      compartment_names <- compartment_names[-1]
    }
    i_mean = mean(sim_df$i)
    gt_mean_days = sim_df[sim_df$i >= i_mean,]$day
    display_day = gt_mean_days[1]
    sim_long_df = gather(data = sim_df, key = "compartment_sim", value = "count",
                         all_of(c('tweets', compartment_names)))
    return(plot + 
             geom_point(data = sim_long_df, aes(y = count, 
                                                color = compartment_sim),
                      size = .5) + 
             geom_label_repel(data = subset(sim_long_df, 
                                            day == display_day), 
                              aes(label = compartment_sim,
                                  color = compartment_sim)))
    
}
